Holoviz
=======

Holoviz is a suite of packages that provide an open-source alternative to the `plotly` suite:

<img src="https://holoviz.org/assets/panel.png" alt="panel" style="width: 50px;"/>: `dash` like library for dashboard generation.

<img src="https://holoviz.org/assets/hvplot.png" alt="hvplot" style="width: 50px;"/>: provides `matplotlib` like plotting API to multiple backend.

<img src="https://holoviz.org/assets/holoviews.png" alt="holoviews" style="width: 50px;"/>: alternative of `hvplot`

<img src="https://holoviz.org/assets/geoviews.png" alt="geoviews" style="width: 50px;"/>: visualization for geo science

<img src="https://holoviz.org/assets/datashader.png" alt="datashader" style="width: 50px;"/>: visualization helper class for large dataset.

<img src="https://holoviz.org/assets/lumen.png" alt="lumen" style="width: 50px;"/>: framework for live data visualizatoin

<img src="https://holoviz.org/assets/param.png" alt="param" style="width: 50px;"/>: light-weight class for element definition

<img src="https://holoviz.org/assets/colorcet.png" alt="colorcet" style="width: 50px;"/>: collection of color maps

In [1]:
import numpy as np
import panel as pn
import param
import holoviews as hv
from holoviews import streams
from holoviews import opts
from holoviews.operation.datashader import rasterize

pn.extension('vtk')
hv.extension("bokeh")

In [2]:
class Sinewave(param.Parameterized):
    amp = param.Number(default=1.0, bounds=(0, None), doc="almplitude")
    phase = param.Number(default=0.0, bounds=(0, np.pi*2), doc="phase")

    @param.depends("amp", "phase")
    def viewer(self):
        x = np.linspace(0, 4*np.pi, 100)
        y = self.amp* np.sin(x + self.phase)
        
        scatter1 = hv.Scatter((x, y), label='Asin(x+b)')
        scatter2 = hv.Scatter((x, y*2), label='2A*sin(x+b)').opts(color='orange')
        scatter3 = hv.Scatter((x, y*3), label='3A*sin(x+b)').opts(color='green')
        scatter4 = hv.Scatter(scatter3).opts(line_color='green', marker='square', fill_alpha=0, size=5)

        curve1 = hv.Curve(scatter1)
        curve2 = hv.Curve(scatter2).opts(line_dash=(4, 4), color='orange')
        curve3 = hv.Curve(scatter3).opts(color='orange')
        
        example1 = scatter1 * scatter2 * scatter3
        example2 = scatter1 *  curve1 * curve2 * scatter4 * curve3
        return example1.relabel("Example") + example2.relabel("Another Example")

    def panel(self):
        amp_input = pn.widgets.FloatInput.from_param(self.param.amp)
        phase_input = pn.widgets.FloatSlider.from_param(self.param.phase)
        return pn.Column(
            amp_input,
            phase_input,
            self.viewer,
        )


In [3]:
app = Sinewave()

In [4]:
app.panel()

Column
    [0] FloatInput(name='Amp', start=0, value=1.0)
    [1] FloatSlider(end=6.283185307179586, name='Phase')
    [2] ParamMethod(method)

we can also start a standalone bokeh server to view the widget

In [ ]:
app.panel().show()

we can also mark the app servable to serve it directly from comand line

```
panel serve 04_holoviz.py
```

It can also provide an easy way to view 3D data from browser via `pyvista` (Python wrapper of `vtk`)

In [5]:
data_matrix = np.zeros([75, 75, 75], dtype=np.uint8)
data_matrix[0:35, 0:35, 0:35] = 50
data_matrix[25:55, 25:55, 25:55] = 100
data_matrix[45:74, 45:74, 45:74] = 150

pn.pane.VTKVolume(data_matrix, sizing_mode='stretch_width', height=400, spacing=(3,2,1), interpolation='nearest', edge_gradient=0, sampling=0)

VTKVolume(ndarray, edge_gradient=0, height=400, interpolation='nearest', sampling=0, sizing_mode='stretch_width', slice_i=37, slice_j=37, slice_k=37, spacing=(3, 2, 1))

now look at something more interesting

In [6]:
from pyvista import examples

# Download a volumetric dataset
vol = examples.download_head()

In [11]:
volume = pn.panel(vol, sizing_mode='stretch_both', height=300, interpolation='nearest')

pn.Row(volume.controls(jslink=True), volume)

Row
    [0] Tabs
        [0] WidgetBox(name='Controls', width=300)
            [0] StaticText(value='<b>Controls</b>')
            [1] DictInput(name='Axes', serializer='json', type=<class 'dict'>)
            [2] DictInput(name='Camera', serializer='json', type=<class 'dict'>)
            [3] ListInput(name='Color mappers', serializer='json', type=<class 'list'>)
            [4] Checkbox(name='Orientation widget')
            [5] Checkbox(disabled=True, name='Interactive o..., value=True)
            [6] FloatInput(name='Ambient', step=0.01, value=0.2)
            [7] Checkbox(name='Controller expanded', value=True)
            [8] Select(name='Colormap', options=OrderedDict([('2hot', ...]), value='erdc_rainbow_bright')
            [9] FloatInput(name='Diffuse', step=0.01, value=0.7)
            [10] Checkbox(name='Display volume', value=True)
            [11] Checkbox(name='Display slices')
            [12] FloatSlider(name='Edge gradient', step=0.01, value=0.4)
            [13] Select(name='Interpolation', options=OrderedDict([('fast_linear...]), value='nearest')
            [14] DictInput(name='Mapper', serializer='json', type=<class 'dict'>)
            [15] FloatSlider(name='Nan opacity', value=1.0)
            [16] ColorPicker(name='Render background', value='#52576e')
            [17] Checkbox(name='Rescale')
            [18] Checkbox(name='Shadow', value=True)
            [19] FloatSlider(name='Sampling', step=0.01, value=0.4)
            [20] FloatInput(name='Specular', step=0.01, value=0.3)
            [21] FloatInput(name='Specular power', value=8.0)
            [22] IntSlider(end=47, name='Slice i', value=23)
            [23] IntSlider(end=61, name='Slice j', value=30)
            [24] IntSlider(end=41, name='Slice k', value=20)
        [1] WidgetBox(name='Layout', width=300)
            [0] StaticText(value='<b>Layout</b>')
            [1] TextInput(disabled=True, name='Name', value='VTKVolume01677')
            [2] LiteralInput(name='Align', serializer='json', value='start')
            [3] LiteralInput(name='Aspect ratio', serializer='json')
            [4] LiteralInput(name='Background', serializer='json')
            [5] ListInput(name='Css classes', serializer='json', type=<class 'list'>)
            [6] IntInput(name='Width', start=0)
            [7] IntInput(name='Height', start=0, value=300)
            [8] IntInput(name='Min width', start=0)
            [9] IntInput(name='Min height', start=0)
            [10] IntInput(name='Max width', start=0)
            [11] IntInput(name='Max height', start=0)
            [12] LiteralInput(name='Margin', serializer='json', value=5)
            [13] Select(name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
            [14] Select(name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
            [15] Select(name='Sizing mode', options=OrderedDict([('fixed', ...]), value='stretch_both')
            [16] Checkbox(name='Visible', value=True)
    [1] VTKVolume(UniformGrid, height=300, interpolation='nearest', origin=(0.0, 0.0, 0.0), sizing_mode='stretch_both', slice_i=23, slice_j=30, slice_k=20, spacing=(4.0, 4.0, 4.0))